In [1]:
# https://github.com/reflash/tinkoff-python-api
# pip install git+https://github.com/GIT_USER_ID/GIT_REPO_ID.git

# https://github.com/TinkoffCreditSystems/invest-openapi
# https://github.com/daxartio/tinvest

# https://tinkoffcreditsystems.github.io/invest-openapi/marketdata/

In [2]:
figi = ''
start_date = '2019-08-19T18:38:33.131642+03:00'
end_date = '2019-08-19T18:38:33.131642+03:00'
interval = 'day'
query = "https://api-invest.tinkoff.ru/openapi/market/candles?figi={0}&from={1}&to={2}&interval={3}".format(figi, start_date, end_date, interval)
query

'https://api-invest.tinkoff.ru/openapi/market/candles?figi=&from=2020-01-01&to=2020-02-01&interval=1min'

In [ ]:
# Ded Inside

In [1]:
k = 'C:/Users/MainUser/Desktop/OpenAPI_sandbox.txt'
crs = open(k, "r")
for columns in (raw.strip().split() for raw in crs):
    token = columns[0]

In [2]:
# https://github.com/Fatal1ty/tinkoff-api


import asyncio
from datetime import datetime
from tinkoff.investments import (
    TinkoffInvestmentsRESTClient, Environment,CandleResolution
)
from tinkoff.investments.client.exceptions import TinkoffInvestmentsError

In [3]:
import pandas

In [4]:
async def show_apple_year_candles(token):
    try:
        async with TinkoffInvestmentsRESTClient(
                # token='TOKEN',
                token = token,
                environment=Environment.SANDBOX) as client:
            
            candles = await client.market.candles.get(
                figi='BBG000B9XRY4',
                dt_from=datetime(2019, 1, 10),
                dt_to=datetime(2019, 1, 12),
                interval=CandleResolution.DAY
            )
            data = []
            print(len(candles))
            for candle in candles:
                data.append(pandas.DataFrame(data=[[candle.time, candle.o, candle.c, candle.h, candle.l, candle.v]]))
                # print(f'{candle.time}: {candle.h}')
            data = pandas.concat(data, axis=0)
            data.columns = ['time', 'open', 'close', 'high', 'low', 'volume']
        return data
    except TinkoffInvestmentsError as e:
        print(e)

In [5]:
oi = await show_apple_year_candles(token=token)  # use this in IPython
# asyncio.run(show_apple_year_candles(token=token))  # use this in plain Python

oi

2


,time,open,close,high,low,volume
0,2019-01-10 07:00:00+00:00,152.69,153.60,153.96,150.89,8993087
0,2019-01-11 07:00:00+00:00,154.82,152.19,154.82,151.57,7650923


In [36]:
async def show_my_year_candles(ticker, token, interval=CandleResolution.MIN_1):
    try:
        async with TinkoffInvestmentsRESTClient(
                # token='TOKEN',
                token = token,
                environment=Environment.SANDBOX) as client:

            instruments = await client.market.instruments.search(ticker)
            instrument = instruments[0]
            figi = instrument.figi
            
            candles = await client.market.candles.get(
                # figi='BBG000B9XRY4',
                # figi=ticker,
                figi=figi,
                dt_from=datetime(2019, 1, 1),
                dt_to=datetime(2019, 12, 31),
                interval=interval
            )
            for candle in candles:
                print(f'{candle.time}: {candle.h}')
    except TinkoffInvestmentsError as e:
        print(e)

In [37]:
await show_my_year_candles(ticker='AAPL', token=token)  # use this in IPython
# asyncio.run(show_my_year_candles(token=token))  # use this in plain Python

VALIDATION_ERROR, [to]: Bad candle interval: from=2019-01-01T00:00:00Z to=2019-12-31T00:00:00Z expected from 1 minute to 1 day


In [40]:
from datetime import timedelta

In [100]:
async def show_my_time_candles(ticker, token, start_date, end_date, interval=CandleResolution.MIN_1):
    try:
        async with TinkoffInvestmentsRESTClient(
                # token='TOKEN',
                token = token,
                environment=Environment.SANDBOX) as client:

            instruments = await client.market.instruments.search(ticker)
            instrument = instruments[0]
            figi = instrument.figi
            
            candles = await client.market.candles.get(
                # figi='BBG000B9XRY4',
                # figi=ticker,
                figi=figi,
                # dt_from=datetime(2019, 1, 1),
                dt_from=start_date,
                # dt_to=datetime(2019, 12, 31),
                dt_to=end_date,
                interval=interval
            )
            data = []
            if len(candles) == 0:
                data = pandas.DataFrame(columns=['time', 'open', 'close', 'high', 'low', 'volume'])
            else:
                for candle in candles:
                    data.append(pandas.DataFrame(data=[[candle.time, candle.o, candle.c, candle.h, candle.l, candle.v]]))
                    # print(f'{candle.time}: {candle.h}')
                data = pandas.concat(data, axis=0)
                data.columns = ['time', 'open', 'close', 'high', 'low', 'volume']
        return data
    except TinkoffInvestmentsError as e:
        print(e)

In [101]:
async def min_partitor(ticker, start_date, end_date, token):
    diff = end_date - start_date
    partitions_needed = diff > timedelta(days=1)
    if partitions_needed:
        result = []
        for dd in range(diff.days):
            # print(dd)
            start_part = start_date + timedelta(days=dd)
            end_part = start_date + timedelta(days=(dd + 1))
            # print(start_part)
            # print(end_part)
            resy = await show_my_time_candles(ticker=ticker, start_date=start_part, end_date=end_part, token=token)
            result.append(resy)
        start_part = start_date + timedelta(days=diff.days)
        end_part = end_date
        if end_part - start_part >= timedelta(minutes=1):
            resy = await show_my_time_candles(ticker=ticker, start_date=start_part, end_date=end_part, token=token)
            result.append(resy)
        result = pandas.concat(result, axis=0)
    else:
        result = await show_my_time_candles(ticker=ticker, start_date=start_date, end_date=end_date, token=token)
    return result

In [96]:
start_date, end_date = datetime(2020, 1, 1), datetime(2020, 3, 1)

result = await min_partitor(ticker='AAPL', start_date=start_date, end_date=end_date, token=token)

0
2020-01-01 00:00:00
2020-01-02 00:00:00
1
2020-01-02 00:00:00
2020-01-03 00:00:00
2
2020-01-03 00:00:00
2020-01-04 00:00:00
3
2020-01-04 00:00:00
2020-01-05 00:00:00
4
2020-01-05 00:00:00
2020-01-06 00:00:00
5
2020-01-06 00:00:00
2020-01-07 00:00:00
6
2020-01-07 00:00:00
2020-01-08 00:00:00
7
2020-01-08 00:00:00
2020-01-09 00:00:00
8
2020-01-09 00:00:00
2020-01-10 00:00:00
9
2020-01-10 00:00:00
2020-01-11 00:00:00
10
2020-01-11 00:00:00
2020-01-12 00:00:00
11
2020-01-12 00:00:00
2020-01-13 00:00:00
12
2020-01-13 00:00:00
2020-01-14 00:00:00
13
2020-01-14 00:00:00
2020-01-15 00:00:00
14
2020-01-15 00:00:00
2020-01-16 00:00:00
15
2020-01-16 00:00:00
2020-01-17 00:00:00
16
2020-01-17 00:00:00
2020-01-18 00:00:00
17
2020-01-18 00:00:00
2020-01-19 00:00:00
18
2020-01-19 00:00:00
2020-01-20 00:00:00
19
2020-01-20 00:00:00
2020-01-21 00:00:00
20
2020-01-21 00:00:00
2020-01-22 00:00:00
21
2020-01-22 00:00:00
2020-01-23 00:00:00
22
2020-01-23 00:00:00
2020-01-24 00:00:00
23
2020-01-24 00:00:0

In [105]:
async def call_them_all(tickers, start_date, end_date, token):

    result = []
    for ticker in tickers:
        resy = await min_partitor(ticker=ticker, start_date=start_date, end_date=end_date, token=token)
        resy['ticker'] = ticker
        result.append(resy)
    result = pandas.concat(result, axis=0)
    
    return result

In [106]:
tickers = ['AAPL', 'MSFT', 'INTC']
start_date, end_date = datetime(2020, 1, 1), datetime(2020, 3, 1)

result = await call_them_all(tickers=tickers, start_date=start_date, end_date=end_date, token=token)

In [107]:
result

,time,open,close,high,low,volume,ticker
0,2020-01-02 07:00:00+00:00,292.52,293.00,293.00,292.42,39,AAPL
0,2020-01-02 07:01:00+00:00,293.00,294.87,294.87,292.99,33,AAPL
0,2020-01-02 07:03:00+00:00,294.88,295.26,295.26,294.88,5,AAPL
0,2020-01-02 07:04:00+00:00,295.26,295.26,295.26,295.26,5,AAPL
0,2020-01-02 07:05:00+00:00,294.88,294.88,294.88,294.88,35,AAPL
...,...,...,...,...,...,...,...
0,2020-02-28 22:37:00+00:00,56.22,56.22,56.22,56.05,286,INTC
0,2020-02-28 22:38:00+00:00,56.22,56.22,56.22,56.22,102,INTC
0,2020-02-28 22:39:00+00:00,56.22,56.22,56.22,56.22,2,INTC
0,2020-02-28 22:41:00+00:00,56.22,56.22,56.22,56.22,1,INTC
